In [1]:
###One-Hot Encoding
import pandas as pd
import numpy as np
import matplotlib.pylab as plt

from datetime import datetime, timedelta, date

from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15, 6

from time import time
import logging

In [2]:
from pyspark.sql import SparkSession

from pyspark.ml import Pipeline

from pyspark.ml.tuning import CrossValidator, ParamGridBuilder, TrainValidationSplit

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import StandardScaler

from pyspark.ml.regression import LinearRegression
from pyspark.sql.types import *
from pyspark.sql.functions import col, udf
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
from pyspark.mllib.evaluation import BinaryClassificationMetrics

from pyspark.mllib.classification import LogisticRegressionWithLBFGS
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.mllib.util import MLUtils

In [3]:
spark
#spark = SparkSession.builder.master("local").enableHiveSupport().getOrCreate()


In [4]:
data = pd.read_csv('titanic-train3.csv',sep=",",encoding='UTF-8')
data.head()



,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,C43,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,C43,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,C43,S


In [5]:
data.isnull().sum()
#data.Age.mean()
#training, test = data.randomSplit([0.6, 0.4], seed=11)


PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

In [6]:

#data.head()

In [7]:
#data.Age.replace(np.NaN , data.Age.mean()).head(15)

In [8]:
schema = StructType([
       StructField("passengerid", IntegerType(), True),
       StructField("survived", IntegerType(), True),
       StructField("pclass", IntegerType(), True),
       StructField("name", StringType(), True),
       StructField("sex", StringType(), True),
       StructField("age", DoubleType(), True),
       StructField("sibsp", IntegerType(), True),
	   StructField("parch", IntegerType(), True),
       StructField("ticket", StringType(), True),
       StructField("fare", FloatType(), True),
       StructField("cabin", StringType(), True),
       StructField("embarked", StringType(), True)])

In [9]:
dataset = spark.createDataFrame(data, schema)
# training, test = dataset.randomSplit([0.6, 0.4], seed=100)
# training.cache()
# print(training.count())
# print(test.count())

In [10]:
#dataset['Age'].replace(np.NaN , dataset.Age.mean()).head(15)

In [11]:
cols = dataset.columns
print cols

['passengerid', 'survived', 'pclass', 'name', 'sex', 'age', 'sibsp', 'parch', 'ticket', 'fare', 'cabin', 'embarked']


In [12]:
from pyspark.ml.feature import Imputer

df = spark.createDataFrame([
    (1.0, float("nan")),
    (2.0, float("nan")),
    (float("nan"), 3.0),
    (4.0, 4.0),
    (5.0, 5.0)
], ["a", "b"])

imputer = Imputer(inputCols=["a", "b"], outputCols=["out_a", "out_b"])
model = imputer.fit(df)

model.transform(df).show()

+---+---+-----+-----+
|  a|  b|out_a|out_b|
+---+---+-----+-----+
|1.0|NaN|  1.0|  4.0|
|2.0|NaN|  2.0|  4.0|
|NaN|3.0|  3.0|  3.0|
|4.0|4.0|  4.0|  4.0|
|5.0|5.0|  5.0|  5.0|
+---+---+-----+-----+



In [13]:
from pyspark.ml.feature import Imputer
stages = []
imputer = Imputer(inputCols=["age"], outputCols=["age"])
model = imputer.fit(dataset)
model.transform(dataset).show()
stages += [imputer]

+-----------+--------+------+--------------------+------+-----+-----+-----+----------------+-------+-----+--------+
|passengerid|survived|pclass|                name|   sex|  age|sibsp|parch|          ticket|   fare|cabin|embarked|
+-----------+--------+------+--------------------+------+-----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male| 22.0|    1|    0|       A/5 21171|   7.25|  C43|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female| 38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female| 26.0|    0|    0|STON/O2. 3101282|  7.925|  C43|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female| 35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male| 35.0|    0|    0|          373450|   8.05|  C43|       S|
|          6|       0|     3|    Moran, Mr. James|  male|29.69|    0|   

In [14]:
dataset.count()

891

In [15]:
'''def dropMissingvalues(dataset):
    dataset=dataset.na.drop()
dropMissingvalues
stages += [dropMissingvalues]'''

'def dropMissingvalues(dataset):\n    dataset=dataset.na.drop()\ndropMissingvalues\nstages += [dropMissingvalues]'

In [16]:
dataset.count()

891

In [17]:
categoricalColumns = ["name","sex","ticket","cabin","embarked"]
#stages = [] # stages in our Pipeline
for categoricalCol in categoricalColumns:
  # Category Indexing with StringIndexer
  stringIndexer = StringIndexer(inputCol=categoricalCol, outputCol=categoricalCol+"Index")
  # Use OneHotEncoder to convert categorical variables into binary SparseVectors
  encoder = OneHotEncoder(inputCol=categoricalCol+"Index", outputCol=categoricalCol+"classVec")
  # Add stages.  These are not run here, but will run all at once later on.
  stages += [stringIndexer, encoder]

In [18]:
#numericCols = ["passengerid","age","pclass", "sibsp", "parch","fare"]
#assemblerInputs = map(lambda c: c + "classVec", categoricalColumns) + numericCols
#assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
#stages += [assembler]

In [19]:
# Convert label into label indices using the StringIndexer
label_stringIdx = StringIndexer(inputCol = "survived", outputCol = "label")
stages += [label_stringIdx]

In [20]:
stages

[Imputer_46829268f49f53e3717e,
 StringIndexer_4ccb95827620d1cdd691,
 OneHotEncoder_4666a51ba2c03f58802e,
 StringIndexer_44c486c8b36f0842a551,
 OneHotEncoder_451ea7329702175aa4c8,
 StringIndexer_400cbe81b4b7b71ff64a,
 OneHotEncoder_43038c9abb32a83bcf69,
 StringIndexer_4bf9a1bd22ac5777227d,
 OneHotEncoder_4885ac74691babcdf383,
 StringIndexer_466a83b7d780a737be3a,
 OneHotEncoder_4cef984dd3f14bce9a21,
 StringIndexer_44ff9b66d869ea6e667a]

In [25]:
from pyspark.ml.classification import LogisticRegression

# Create initial LogisticRegression model

lr = LogisticRegression( maxIter=2)

#labelCol="label", featuresCol="features"

In [26]:
stages += [lr]

In [27]:
# stages
dataset.head(5)

[Row(passengerid=1, survived=0, pclass=3, name=u'Braund, Mr. Owen Harris', sex=u'male', age=22.0, sibsp=1, parch=0, ticket=u'A/5 21171', fare=7.25, cabin=u'C43', embarked=u'S'),
 Row(passengerid=2, survived=1, pclass=1, name=u'Cumings, Mrs. John Bradley (Florence Briggs Thayer)', sex=u'female', age=38.0, sibsp=1, parch=0, ticket=u'PC 17599', fare=71.2833023071289, cabin=u'C85', embarked=u'C'),
 Row(passengerid=3, survived=1, pclass=3, name=u'Heikkinen, Miss. Laina', sex=u'female', age=26.0, sibsp=0, parch=0, ticket=u'STON/O2. 3101282', fare=7.925000190734863, cabin=u'C43', embarked=u'S'),
 Row(passengerid=4, survived=1, pclass=1, name=u'Futrelle, Mrs. Jacques Heath (Lily May Peel)', sex=u'female', age=35.0, sibsp=1, parch=0, ticket=u'113803', fare=53.099998474121094, cabin=u'C123', embarked=u'S'),
 Row(passengerid=5, survived=0, pclass=3, name=u'Allen, Mr. William Henry', sex=u'male', age=35.0, sibsp=0, parch=0, ticket=u'373450', fare=8.050000190734863, cabin=u'C43', embarked=u'S')]

In [28]:
# Create a Pipeline.
pipeline = Pipeline(stages=stages)
# Run the feature transformations.
#  - fit() computes feature statistics as needed.
#  - transform() actually transforms the features.
pipelineModel = pipeline.fit(dataset)
prediction = pipelineModel.transform(dataset)

# Keep relevant columns
# selectedcols = ["label", "features"] + cols
# dataset = dataset.select(cols)
print(dataset)

IllegalArgumentException: u'Field "features" does not exist.\nAvailable fields: passengerid, survived, pclass, name, sex, age, sibsp, parch, ticket, fare, cabin, embarked, nameIndex, nameclassVec, sexIndex, sexclassVec, ticketIndex, ticketclassVec, cabinIndex, cabinclassVec, embarkedIndex, embarkedclassVec, label'

In [ ]:
predictionAndLabels = prediction.select("prediction","survived")
predictionAndLabels.show()

In [ ]:
predictionAndLabels = predictionAndLabels.selectExpr("prediction", "survived as label")
predictionAndLabels.show()

In [ ]:
predictionAndLabels.show()

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator(rawPredictionCol="prediction")
evaluator.setMetricName("areaUnderPR")
evaluator.evaluate(predictionAndLabels)

In [ ]:
evaluator.getMetricName()


In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator(rawPredictionCol="prediction")
evaluator.setMetricName("areaUnderROC")
evaluator.evaluate(predictionAndLabels)

In [ ]:
evaluator.getMetricName()

In [ ]:
pipelineModel.write().overwrite().save("hdfs://inhyd-bdamapr-1.lab.opentext.com:7222/home/opentext/bda/ML_Model/LRB")